In [1]:
import tensorflow as tf
import pickle
import os
import shutil
import tempfile
import datetime
from glob import iglob
import time
# import attention
from collections import deque
import pickle
import pyreader
import numpy as np
import sys
import csv
from batchmake import Batcher
import astwalker
import ast
import tokenize
from io import StringIO

In [2]:
pad_token, pad_id = "§PAD§", 0
oov_token, oov_id = "§OOV§", 1
indent_token = "§<indent>§"
dedent_token = "§<dedent>§"
number_token = "§NUM§"

In [3]:
datas= 'data_samples/'
word_to_id='data_samples/mapping.map'
with open(word_to_id, 'rb') as f:
    word_to_id = pickle.load(f)
inv_map = {v:k for k, v in word_to_id.items() }
# batch_data = datas+'output.txt.part0'
batch_data = datas+'preprocess.part0'
with open(batch_data, 'rb') as f:
    picked_data = pickle.load(f)
    
print(len(picked_data))
train_proportion = 0.5
valid_proportion = 0.2

481


In [4]:
subdirectories = [os.path.join(datas,o) for o in os.listdir(datas) if os.path.isdir(os.path.join(datas,o))]

In [5]:
python_files = [(directory, [y for x in os.walk(directory) for y in iglob(os.path.join(x[0], '*.py'))])
                for directory in subdirectories]

In [6]:
train_split = int(len(python_files) * train_proportion)
valid_split = train_split + int(len(python_files) * valid_proportion)
train_files = []
valid_files = []
test_files = []

In [7]:
for project in python_files[:train_split]:
    train_files.extend([f[len(datas):] for f in project[1]])
for project in python_files[train_split:valid_split]:
    valid_files.extend([f[len(datas):] for f in project[1]])
for project in python_files[valid_split:]:
    test_files.extend([f[len(datas):] for f in project[1]])

In [8]:
def write_to_file(fname, lst):
    with open(os.path.join(datas, fname), "w") as write_file:
        for f in lst:
            print(f, file=write_file)

In [9]:
write_to_file("train_files.txt", train_files)
write_to_file("valid_files.txt", valid_files)
write_to_file("test_files.txt", test_files)

### 데이터셋을 train, valid, test셋으로 만든다.

### 파일 읽기

In [10]:
python_files = [os.path.join(datas, f) for f in train_files]
mapping = (lambda x: x) if word_to_id is None else (lambda x: word_to_id.get(x, oov_id))
def get_source_tree(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        fstr = f.read()
    fstr = fstr.replace('\r\n', '\n').replace('\r', '\n')
    if not fstr.endswith('\n'):
        fstr += '\n'
    return fstr, ast.parse(fstr, filename=filename)

In [11]:
def preprocess(tokentype, tokenval):
    if tokentype == tokenize.NUMBER:
        return number_token

    elif tokentype == tokenize.INDENT:
        return indent_token

    elif tokentype == tokenize.DEDENT:
        return dedent_token

    return tokenval

In [12]:
data = []
definition_positions = []
identifier_usage = []
gen_def_positions = True
error_data = 0
data_file_name = []
for filename in python_files:
    try:
        source, tree = get_source_tree(filename)
        tokens = tokenize.generate_tokens(StringIO(source).readline)
        data.append([(mapping(preprocess(tokenType, tokenVal)), start) for tokenType, tokenVal, start, _, _
                     in tokens
                     if tokenType != tokenize.COMMENT and
                     not tokenVal.startswith("'''") and
                     not tokenVal.startswith('"""') and 
                     (tokenType == tokenize.DEDENT or tokenVal != "")])
        data_file_name.append(filename)
        if gen_def_positions:
            walker = astwalker.ASTWalker()
            walker.walk(tree)
            definition_positions.append(walker.definition_positions)
            identifier_usage.append(walker.name_usage)
    except:
        error_data+=1
        continue
        print("Error when tokenizing %s: %s" % (filename, sys.exc_info()[:3]))

### 전체 파일 ( python_files ) 중에서 에러난거 뺀데이터 (data) 

In [13]:
len(python_files),error_data,len(data), len(definition_positions)

(534, 55, 479, 479)

### 데이터 비교

In [14]:
d , dd=get_source_tree(data_file_name[0])
print(d)

from __future__ import unicode_literals
import re
import ast
try:
    from setuptools import setup
except ImportError:
    from distutils.core import setup

def function2835():
    with open('zhihu/__init__.py', 'rb') as var2990:
        var4009 = re.search('__version__ = (.*)', var2990.read().decode('utf-8')).group(1)
        if (var4009 is None):
            raise RuntimeError('Cannot find version information')
        return str(ast.literal_eval(var4009))
with open('README.rst', 'rb') as var2178:
    var2523 = var2178.read().decode('utf-8')
var4384 = ['zhihu']
var3646 = function2835()
setup(name='zhihu-py3', version=var3646, keywords=['zhihu', 'network', 'spider', 'html'], description='Zhihu UNOFFICIAL API library in python3, with help of bs4, lxml, requests and html2text.', long_description=var2523, author='7sDream', author_email='didislover@gmail.com', license='MIT', url='https://github.com/7sDream/zhihu-py3', download_url='https://github.com/7sDream/zhihu-py3', install_requires=[

In [15]:
token_to_str = []
for tokend_data,position in data[0]:
     token_to_str.append(inv_map[tokend_data])
print(' '.join(token_to_str))

from __future__ import unicode_literals 
 import re 
 import §OOV§ 
 try : 
 §<indent>§ from §OOV§ import setup 
 §<dedent>§ except ImportError : 
 §<indent>§ from §OOV§ . §OOV§ import setup 
 
 §<dedent>§ def function2835 ( ) : 
 §<indent>§ with open ( §OOV§ , 'rb' ) as var2990 : 
 §<indent>§ var4009 = re . search ( §OOV§ , var2990 . read ( ) . decode ( 'utf-8' ) ) . group ( §NUM§ ) 
 if ( var4009 is None ) : 
 §<indent>§ raise RuntimeError ( §OOV§ ) 
 §<dedent>§ return str ( §OOV§ . §OOV§ ( var4009 ) ) 
 §<dedent>§ §<dedent>§ with open ( §OOV§ , 'rb' ) as var2178 : 
 §<indent>§ var2523 = var2178 . read ( ) . decode ( 'utf-8' ) 
 §<dedent>§ var4384 = [ §OOV§ ] 
 var3646 = function2835 ( ) 
 setup ( name = 'zhihu-py3' , version = var3646 , §OOV§ = [ §OOV§ , §OOV§ , §OOV§ , 'html' ] , description = §OOV§ , §OOV§ = var2523 , author = '7sDream' , §OOV§ = §OOV§ , §OOV§ = §OOV§ , url = §OOV§ , §OOV§ = §OOV§ , §OOV§ = [ §OOV§ , §OOV§ , §OOV§ ] , §OOV§ = { §OOV§ : [ §OOV§ ] , } , packages = v

### 원본데이터와 토큰데이터

In [16]:
type_max_rands = {
    "var": 4750, "function": 2900, "Class": 440, "attribute": 2400, "arg": 2400
}

identifier_types = [key for key in type_max_rands]
def_positions = [[[t[1] for t in fp if t[0] == k] for k in identifier_types] for fp in definition_positions]
# ex) def_positions[101] #function, var, class, attribute, arg

In [17]:
inputs = []
for line_data in data:
    line_data_list = []
    for i, token in enumerate(line_data):
#         print(token[0])
#         inputs.append(token[0])
        line_data_list.append(token[0])
        if i == 10:
            inputs.append(line_data_list)
            line_data_list=[]
            
#             print(token[0])
        

In [18]:
for row in inputs[:30]:
    print(row)

[38, 455, 19, 3138, 2, 19, 63, 2, 19, 1, 2]
[19, 62, 2, 19, 42, 2, 62, 5, 50, 5, 1]
[38, 455, 19, 794, 6, 1720, 6, 3138, 2, 19, 273]
[38, 455, 19, 794, 6, 1720, 6, 3138, 2, 19, 273]
[38, 455, 19, 794, 6, 1720, 6, 3138, 2, 19, 273]
[38, 455, 19, 794, 6, 1720, 6, 3138, 2, 19, 273]
[19, 273, 2, 19, 63, 2, 38, 1008, 5, 637, 19]
[38, 455, 19, 794, 6, 1720, 6, 3138, 2, 19, 273]
[38, 455, 19, 794, 6, 1720, 6, 3138, 2, 19, 273]
[19, 42, 2, 19, 1, 2, 2, 17, 3722, 3, 7018]
[6568, 7, 4247, 2, 19, 42, 2, 19, 245, 2, 38]
[38, 74, 19, 74, 2, 38, 5, 1, 19, 456, 2]
[19, 1, 2, 1193, 7, 35, 1, 8, 1, 6, 1]
[19, 148, 2, 38, 74, 19, 74, 2, 38, 5, 637]
[19, 148, 2, 38, 5, 2216, 19, 1017, 2, 38, 5]
[38, 5, 637, 19, 260, 2, 38, 2468, 19, 2599, 2]
[38, 5, 637, 19, 22, 2, 38, 5, 2216, 19, 1017]
[38, 5, 637, 19, 22, 2, 38, 5, 2216, 19, 1017]
[56, 7618, 8, 2, 9, 1, 2, 2, 17, 65, 3]
[19, 148, 2, 38, 5, 637, 19, 22, 2, 38, 5]
[38, 5, 637, 19, 22, 2, 38, 5, 2216, 19, 1017]
[19, 148, 2, 19, 126, 2, 38, 74, 19, 74, 2]

In [19]:
with open('inputs.csv', 'w',newline='', encoding='utf-8') as csvfile:
    spamwriter = csv.writer(csvfile)
    for row_line in inputs:
        spamwriter.writerow(row_line)

### target은 input+1 의 인덱스로 하고, sequence는 실행시간에 정함.